#Section 0: Setup

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%writefile read_input.h
#include <stdio.h>
#include <stdlib.h>

int read_input_one_two_four(int **input1, char* filepath) {
 FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    }

    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));

    int temp1;

    while (fscanf(fp, "%d", &temp1) == 1) {
        (*input1)[counter] = temp1;

        counter++;
    }

    fclose(fp);
    return len;




}
int read_input_three(int** input1, int** input2, int** input3, int** input4,char* filepath){
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    }

    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));
    *input2 = ( int *)malloc(len * sizeof(int));
    *input3 = ( int *)malloc(len * sizeof(int));
    *input4 = ( int *)malloc(len * sizeof(int));



    int temp1;
    int temp2;
    int temp3;
    int temp4;
    while (fscanf(fp, "%d,%d,%d,%d", &temp1, &temp2, &temp3, &temp4) == 4) {
        (*input1)[counter] = temp1;
        (*input2)[counter] = temp2;
        (*input3)[counter] = temp3;
        (*input4)[counter] = temp4;
        counter++;
    }

    fclose(fp);
    return len;

}

void write_output_file(const char *filename, int *data, int size) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        perror("Error opening output file");
        exit(EXIT_FAILURE);
    }
    fprintf(file, "%d\n", size);
    for (int i = 0; i < size; i++) {
        fprintf(file, "%d\n", data[i]);
    }
    fclose(file);
}


Writing read_input.h


# Section 1: Sequential

In [4]:
%%writefile sequential.c
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <time.h>  // For measuring execution time


#define MAX_NODES 1000000  // Adjust based on expected graph size
#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define XNOR 5

// Gate Execution Function
int execute(int a, int b, int gate) {
    switch(gate) {
        case AND: return a & b;
        case OR: return a | b;
        case NAND: return (a & b) == 0;
        case NOR: return (a | b) == 0;
        case XOR: return a ^ b;
        case XNOR: return (a ^ b) == 0;
        default: return 0;
    }
}

// BFS Traversal
void bfs(int *nodePtrs, int *nodeNeighbors, int *nodeVisited, int *nodeOutput,
         int *nodeGate, int *nodeInput, int *currLevelNodes, int numCurrLevelNodes,
         int *nextLevelNodes, int *numNextLevelNodes) {
    *numNextLevelNodes = 0;

    for (int idx = 0; idx < numCurrLevelNodes; idx++) {
        int node = currLevelNodes[idx];

        // Iterate over all neighbors
        for (int nbrIdx = nodePtrs[node]; nbrIdx < nodePtrs[node + 1]; nbrIdx++) {
            int neighbor = nodeNeighbors[nbrIdx];

            // If the neighbor hasn't been visited yet
            if (!nodeVisited[neighbor]) {
                nodeVisited[neighbor] = 1;  // Mark as visited

                // Compute output using execute function
                nodeOutput[neighbor] = execute(nodeOutput[node], nodeInput[neighbor], nodeGate[neighbor]);
                //printf("Node %d input 1: %d input 2:  %d gate: %d output: %d\n", neighbor, nodeOutput[node], nodeInput[neighbor], nodeGate[neighbor], nodeOutput[neighbor]);
                // Add neighbor to next level queue
                nextLevelNodes[*numNextLevelNodes] = neighbor;
                (*numNextLevelNodes)++;
            }
        }
    }
}

// Function to read single-column input
int read_input_one_two_four(int **data, char* filepath) {
    FILE* fp = fopen(filepath, "r");
    if (!fp) {
        fprintf(stderr, "Couldn't open file %s for reading\n", filepath);
        exit(EXIT_FAILURE);
    }

    int len;
    if (fscanf(fp, "%d", &len) != 1) {
        fprintf(stderr, "Error reading size from %s\n", filepath);
        fclose(fp);
        exit(EXIT_FAILURE);
    }

    *data = (int *)malloc(len * sizeof(int));
    if (!*data) {
        perror("Memory allocation failed");
        fclose(fp);
        exit(EXIT_FAILURE);
    }

    for (int i = 0; i < len; i++) {
        if (fscanf(fp, "%d", &((*data)[i])) != 1) {
            fprintf(stderr, "Error reading data from %s\n", filepath);
            fclose(fp);
            exit(EXIT_FAILURE);
        }
    }

    fclose(fp);
    return len;
}

// Function to read four-column input
int read_input_three(int **input1, int **input2, int **input3, int **input4, char* filepath) {
    FILE* fp = fopen(filepath, "r");
    if (!fp) {
        fprintf(stderr, "Couldn't open file %s for reading\n", filepath);
        exit(EXIT_FAILURE);
    }

    int len;
    if (fscanf(fp, "%d", &len) != 1) {
        fprintf(stderr, "Error reading size from %s\n", filepath);
        fclose(fp);
        exit(EXIT_FAILURE);
    }

    *input1 = (int *)malloc(len * sizeof(int));
    *input2 = (int *)malloc(len * sizeof(int));
    *input3 = (int *)malloc(len * sizeof(int));
    *input4 = (int *)malloc(len * sizeof(int));

    if (!*input1 || !*input2 || !*input3 || !*input4) {
        perror("Memory allocation failed");
        fclose(fp);
        exit(EXIT_FAILURE);
    }

    for (int i = 0; i < len; i++) {
        int read = fscanf(fp, "%d,%d,%d,%d", &((*input1)[i]), &((*input2)[i]), &((*input3)[i]), &((*input4)[i]));
        if (read != 4) {
            fprintf(stderr, "Error reading line %d in %s. Read %d values instead of 4.\n", i, filepath, read);
            exit(EXIT_FAILURE);
        }
    }

    fclose(fp);
    return len;
}

// Function to write output to a file
void write_output_file(const char *filename, int *data, int size) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        perror("Error opening output file");
        exit(EXIT_FAILURE);
    }
    fprintf(file, "%d\n", size);
    for (int i = 0; i < size; i++) {
        fprintf(file, "%d\n", data[i]);
    }
    fclose(file);
}

// Main Function
int main(int argc, char *argv[]) {
    if (argc != 7) {
        fprintf(stderr, "Usage: %s <input1.raw> <input2.raw> <input3.raw> <input4.raw> <output1.raw> <output2.raw>\n", argv[0]);
        return EXIT_FAILURE;
    }

    // Variables
    int numNodePtrs;
    int numNodes;
    int *nodePtrs_h;
    int *nodeNeighbors_h;
    int *nodeVisited_h;
    int numTotalNeighbors_h;
    int *currLevelNodes_h;
    int numCurrLevelNodes;
    int *nodeGate_h;
    int *nodeInput_h;
    int *nodeOutput_h;

    // Read input files
    numNodePtrs = read_input_one_two_four(&nodePtrs_h, argv[1]);
    numTotalNeighbors_h = read_input_one_two_four(&nodeNeighbors_h, argv[2]);
    numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, argv[3]);
    numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, argv[4]);

    // Allocate memory for BFS
    int *nextLevelNodes = (int *)malloc(numNodes * sizeof(int));
    if (!nextLevelNodes) {
        perror("Memory allocation failed for nextLevelNodes");
        exit(EXIT_FAILURE);
    }

    int numNextLevelNodes = 0;

    // Start timing
    clock_t start, end;
    double cpu_time_used;
    start = clock();

    // Perform BFS traversal
    bfs(nodePtrs_h, nodeNeighbors_h, nodeVisited_h, nodeOutput_h, nodeGate_h, nodeInput_h,
        currLevelNodes_h, numCurrLevelNodes, nextLevelNodes, &numNextLevelNodes);

    end = clock();
    cpu_time_used = ((double)(end - start)) / CLOCKS_PER_SEC * 1000;

    printf("Sequential BFS Execution Time: %f milliseconds\n", cpu_time_used);


    // Write outputs
    write_output_file(argv[5], nodeOutput_h, numNodes);
    write_output_file(argv[6], nextLevelNodes, numNextLevelNodes);

    // Free allocated memory
    free(nodePtrs_h);
    free(nodeNeighbors_h);
    free(nodeVisited_h);
    free(nodeGate_h);
    free(nodeInput_h);
    free(nodeOutput_h);
    free(currLevelNodes_h);
    free(nextLevelNodes);

    return EXIT_SUCCESS;
}

Writing sequential.c


In [5]:
!gcc sequential.c -o sequential

In [6]:
!head /content/drive/MyDrive/ECSE420/lab3/input3.raw

200000
0,4,1,-1
0,5,1,-1
0,0,1,-1
0,2,1,-1
0,1,1,-1
0,0,0,-1
0,4,1,-1
1,3,1,0
0,2,1,-1


In [7]:
!rm -r /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput.raw
!rm -r /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes.raw

!./sequential /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes.raw

Sequential BFS Execution Time: 2.017000 milliseconds


In [8]:
!gcc /content/drive/MyDrive/ECSE420/lab3/compareNextLevelNodes.c -o compareNextLevelNodes

In [9]:
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw

No errors!


In [10]:
!gcc /content/drive/MyDrive/ECSE420/lab3/compareNodeOutput.c -o compareNodeOutput

In [11]:
!head /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

==> /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput.raw <==
200000
1
0
1
1
1
0
0
0
1

==> /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw <==
200000
1
0
1
1
1
0
0
0
1


In [12]:
!tail /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

==> /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput.raw <==
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1

==> /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw <==
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [13]:
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

Total Errors : 0	

#Section 2: Global Queueing


In [14]:
%%writefile global_queueing.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define MAX_NODES 1000000
#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define XNOR 5

// Device Function: Gate Execution
__device__ int execute(int a, int b, int gate) {
    int result;
    switch(gate) {
        case AND: result = a & b; break;
        case OR: result = a | b; break;
        case NAND: result = (a & b) == 0; break;
        case NOR: result = (a | b) == 0; break;
        case XOR: result = a ^ b; break;
        case XNOR: result = (a ^ b) == 0; break;
        default: result = 0;
    }

    if (result == -1) {
        printf("WARNING: execute() returned -1 for Inputs %d, %d, Gate %d\n", a, b, gate);
    }
    return result;
}
/*
__global__ void bfs_kernel(int *nodePtrs, int *nodeNeighbors, int *nodeVisited,
                           int *nodeOutput, int *nodeGate, int *nodeInput,
                           int *currLevelNodes, int numCurrLevelNodes,
                           int *nextLevelNodes, int *numNextLevelNodes) {


                          int idx = blockIdx.x * blockDim.x + threadIdx.x; // global thread index that corresponds to current node

                          if (threadIdx.x == 0 && blockIdx.x == 0) {
                              printf("\n");
                              printf("CUDA Kernel Executing with numCurrLevelNodes = %d, Total Threads = %d\n",
                              numCurrLevelNodes, gridDim.x * blockDim.x);
                          }

                          //if (threadIdx.x == 0 && blockIdx.x == 0) {
                            //for (int i = 0; i < 10; i++) {
                            //    printf("GPU Output[%d] Before Copy: %d\n", i, nodeOutput[i]);
                           // }
                          //}

                          if (idx >= numCurrLevelNodes) return; //check that we are within the boundaries
                          for (int i = idx; i < numCurrLevelNodes; i += gridDim.x * blockDim.x) {
                            int node = currLevelNodes[i]; // find responsible node

                            for (int nbrIdx = nodePtrs[node]; nbrIdx < nodePtrs[node + 1]; nbrIdx++) { //loop over all the neighbors
                                int neighbor = nodeNeighbors[nbrIdx];
                                if (atomicCAS(&nodeVisited[neighbor], 0, 1) == 0) {
                                  nodeOutput[neighbor] = execute(nodeOutput[node], nodeInput[neighbor], nodeGate[neighbor]);

                                  int nextIdx = atomicAdd(numNextLevelNodes, 1);
                                  nextLevelNodes[nextIdx] = neighbor;


                                }

                                }
                                }
}
*/
__global__ void bfs_kernel(int *nodePtrs, int *nodeNeighbors, int *nodeVisited,
                           int *nodeOutput, int *nodeGate, int *nodeInput,
                           int *currLevelNodes, int numCurrLevelNodes,
                           int *nextLevelNodes, int *numNextLevelNodes) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;


    if (idx >= numCurrLevelNodes) return; // Bounds check

    for (int i = idx; i < numCurrLevelNodes; i += gridDim.x * blockDim.x) {
        int node = currLevelNodes[i]; // Responsible node

        for (int nbrIdx = nodePtrs[node]; nbrIdx < nodePtrs[node + 1]; nbrIdx++) { // Loop over neighbors
            int neighbor = nodeNeighbors[nbrIdx];

            if (atomicCAS(&nodeVisited[neighbor], 0, 1) == 0) { // Ensure exclusive access
                // Atomic update of node output
                atomicExch(&nodeOutput[neighbor], execute(nodeOutput[node], nodeInput[neighbor], nodeGate[neighbor]));

                // Atomically add to next level queue
                int nextIdx = atomicAdd(numNextLevelNodes, 1);
                nextLevelNodes[nextIdx] = neighbor;

                __threadfence(); // Memory consistency barrier
            }
        }


    }
}


// Function to read single-column input
int read_input_one_two_four(int **data, char* filepath) {
    FILE* fp = fopen(filepath, "r");
    if (!fp) {
        fprintf(stderr, "Couldn't open file %s for reading\n", filepath);
        exit(EXIT_FAILURE);
    }

    int len;
    if (fscanf(fp, "%d", &len) != 1) {
        fprintf(stderr, "Error reading size from %s\n", filepath);
        fclose(fp);
        exit(EXIT_FAILURE);
    }

    *data = (int *)malloc(len * sizeof(int));
    if (!*data) {
        perror("Memory allocation failed");
        fclose(fp);
        exit(EXIT_FAILURE);
    }

    for (int i = 0; i < len; i++) {
        if (fscanf(fp, "%d", &((*data)[i])) != 1) {
            fprintf(stderr, "Error reading data from %s\n", filepath);
            fclose(fp);
            exit(EXIT_FAILURE);
        }
    }

    fclose(fp);
    return len;
}

// Function to read four-column input
int read_input_three(int **input1, int **input2, int **input3, int **input4, char* filepath) {
    FILE* fp = fopen(filepath, "r");
    if (!fp) {
        fprintf(stderr, "Couldn't open file %s for reading\n", filepath);
        exit(EXIT_FAILURE);
    }

    int len;
    if (fscanf(fp, "%d", &len) != 1) {
        fprintf(stderr, "Error reading size from %s\n", filepath);
        fclose(fp);
        exit(EXIT_FAILURE);
    }

    *input1 = (int *)malloc(len * sizeof(int));
    *input2 = (int *)malloc(len * sizeof(int));
    *input3 = (int *)malloc(len * sizeof(int));
    *input4 = (int *)malloc(len * sizeof(int));

    if (!*input1 || !*input2 || !*input3 || !*input4) {
        perror("Memory allocation failed");
        fclose(fp);
        exit(EXIT_FAILURE);
    }

    for (int i = 0; i < len; i++) {
        int read = fscanf(fp, "%d,%d,%d,%d", &((*input1)[i]), &((*input2)[i]), &((*input3)[i]), &((*input4)[i]));
        if (read != 4) {
            fprintf(stderr, "Error reading line %d in %s. Read %d values instead of 4.\n", i, filepath, read);
            exit(EXIT_FAILURE);
        }
    }

    fclose(fp);
    return len;
}

// Function to write output to a file
void write_output_file(const char *filename, int *data, int size) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        perror("Error opening output file");
        exit(EXIT_FAILURE);
    }
    fprintf(file, "%d\n", size);
    for (int i = 0; i < size; i++) {
        fprintf(file, "%d\n", data[i]);
    }
    fclose(file);
}


int main(int argc, char *argv[]) {
    if (argc != 9) {
        fprintf(stderr, "Usage: %s <input1.raw> <input2.raw> <input3.raw> <input4.raw> <output1.raw> <output2.raw> <block_index> <thread_index>\n", argv[0]);
        return EXIT_FAILURE;
    }

    // Host Variables
    int numNodePtrs;
    int numNodes;
    int *nodePtrs_h;
    int *nodeNeighbors_h;
    int *nodeVisited_h;
    int numTotalNeighbors_h;
    int *currLevelNodes_h;
    int numCurrLevelNodes; // Should be an int, NOT a pointer
    int *nodeGate_h;
    int *nodeInput_h;
    int *nodeOutput_h;

    // Read Input Files (on CPU)
    numNodePtrs = read_input_one_two_four(&nodePtrs_h, argv[1]);
    numTotalNeighbors_h = read_input_one_two_four(&nodeNeighbors_h, argv[2]);
    numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, argv[3]);
    numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, argv[4]);
    int block_index = atoi(argv[7]);
    int thread_index = atoi(argv[8]);

    // Allocate memory for BFS
    int *nextLevelNodes_h = (int *)malloc(numNodes * sizeof(int));
    if (!nextLevelNodes_h) {
        perror("Memory allocation failed for nextLevelNodes");
        exit(EXIT_FAILURE);
    }

    int numNextLevelNodes_h = 0;

    // Device Variables
    int *d_nodePtrs, *d_nodeNeighbors, *d_nodeVisited, *d_nodeOutput, *d_nodeGate, *d_nodeInput;
    int *d_currLevelNodes, *d_nextLevelNodes, *d_numNextLevelNodes;

    // Allocate memory on GPU
    cudaMalloc(&d_nodePtrs, numNodePtrs * sizeof(int));
    cudaMalloc(&d_nodeNeighbors, numTotalNeighbors_h * sizeof(int));
    cudaMalloc(&d_nodeVisited, numNodes * sizeof(int));
    cudaMalloc(&d_nodeOutput, numNodes * sizeof(int));
    cudaMalloc(&d_nodeGate, numNodes * sizeof(int));
    cudaMalloc(&d_nodeInput, numNodes * sizeof(int));
    cudaMalloc(&d_currLevelNodes, numCurrLevelNodes * sizeof(int));
    cudaMalloc(&d_nextLevelNodes, numNodes * sizeof(int));
    cudaMalloc(&d_numNextLevelNodes, sizeof(int));

    // Copy data from host to device
    cudaError_t err = cudaMemcpy(d_nodePtrs, nodePtrs_h, numNodePtrs * sizeof(int), cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }
    err = cudaMemcpy(d_nodeNeighbors, nodeNeighbors_h, numTotalNeighbors_h * sizeof(int), cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }
    err = cudaMemcpy(d_nodeVisited, nodeVisited_h, numNodes * sizeof(int), cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }
    err = cudaMemcpy(d_nodeOutput, nodeOutput_h, numNodes * sizeof(int), cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }
    err = cudaMemcpy(d_nodeGate, nodeGate_h, numNodes * sizeof(int), cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }
    err = cudaMemcpy(d_nodeInput, nodeInput_h, numNodes * sizeof(int), cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }
    err = cudaMemcpy(d_currLevelNodes, currLevelNodes_h, numCurrLevelNodes * sizeof(int), cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }
    err = cudaMemset(d_numNextLevelNodes, 0, sizeof(int));
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }

    int numValuesToCheck = 10;
    int *tempArray = (int *)malloc(numValuesToCheck * sizeof(int));

    cudaMemcpy(tempArray, d_nodeOutput, numValuesToCheck * sizeof(int), cudaMemcpyDeviceToHost);

    int threadsPerBlock[] = {32,64,128};  // Can experiment with 32, 64, 128
    int maxBlocks[] = {10, 25, 35};  // Lab-specified limits
    //int block_index = 2;  // Choose 0 for 5 blocks, 1 for 10 blocks, 2 for 25 blocks
    //int thread_index = 2;
    int blocksPerGrid = ((numCurrLevelNodes + threadsPerBlock[thread_index] - 1) / threadsPerBlock[thread_index]);
    blocksPerGrid = (blocksPerGrid < maxBlocks[block_index]) ? blocksPerGrid : maxBlocks[block_index];

    // CUDA Event Timers
    cudaEvent_t start, stop;
    float milliseconds = 0;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);


    // Launch Kernel with Timing
    cudaEventRecord(start);

    bfs_kernel<<<blocksPerGrid, threadsPerBlock[thread_index]>>>(
    d_nodePtrs, d_nodeNeighbors, d_nodeVisited,
    d_nodeOutput, d_nodeGate, d_nodeInput,
    d_currLevelNodes, numCurrLevelNodes,
    d_nextLevelNodes, d_numNextLevelNodes);

    cudaDeviceSynchronize();  // Ensure GPU work is finished before checking


    cudaEventRecord(stop);
    cudaEventSynchronize(stop);


    // Print GPU output after kernel execution


    cudaMemcpy(tempArray, d_nodeOutput, numValuesToCheck * sizeof(int), cudaMemcpyDeviceToHost);


    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Execution Time for %d threads and %d blocks: %f ms\n",threadsPerBlock[thread_index] ,maxBlocks[block_index], milliseconds);

    // Copy results back to host
    err = cudaMemcpy(nextLevelNodes_h, d_nextLevelNodes, numNodes * sizeof(int), cudaMemcpyDeviceToHost);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }
    err = cudaMemcpy(&numNextLevelNodes_h, d_numNextLevelNodes, sizeof(int), cudaMemcpyDeviceToHost);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }
    err = cudaMemcpy(nodeOutput_h, d_nodeOutput, numNodes * sizeof(int), cudaMemcpyDeviceToHost);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }
    err = cudaMemcpy(nodeVisited_h, d_nodeVisited, numNodes * sizeof(int), cudaMemcpyDeviceToHost);
    if (err != cudaSuccess) {
        printf("CUDA memcpy error: %s\n", cudaGetErrorString(err));
    }





    // Write output files
    write_output_file(argv[5], nodeOutput_h, numNodes);
    write_output_file(argv[6], nextLevelNodes_h, numNextLevelNodes_h);

    // Free Device Memory
    cudaFree(d_nodePtrs);
    cudaFree(d_nodeNeighbors);
    cudaFree(d_nodeVisited);
    cudaFree(d_nodeOutput);
    cudaFree(d_nodeGate);
    cudaFree(d_nodeInput);
    cudaFree(d_currLevelNodes);
    cudaFree(d_nextLevelNodes);
    cudaFree(d_numNextLevelNodes);

    // Free Host Memory
    free(nodePtrs_h);
    free(nodeNeighbors_h);
    free(nodeVisited_h);
    free(nodeGate_h);
    free(nodeInput_h);
    free(nodeOutput_h);
    free(currLevelNodes_h);
    free(nextLevelNodes_h);

    return EXIT_SUCCESS;
}




Writing global_queueing.cu


In [15]:
# 128 35 3.703ms

In [16]:
!nvcc --ptxas-options=-v -arch=sm_75 -gencode=arch=compute_75,code=sm_75 global_queueing.cu -o global_queueing
!gcc /content/drive/MyDrive/ECSE420/lab3/compareNextLevelNodes.c -o compareNextLevelNodes
!gcc /content/drive/MyDrive/ECSE420/lab3/compareNodeOutput.c -o compareNodeOutput

ptxas info    : 59 bytes gmem, 16 bytes cmem[4]
ptxas info    : Compiling entry function '_Z10bfs_kernelPiS_S_S_S_S_S_iS_S_' for 'sm_75'
ptxas info    : Function properties for _Z10bfs_kernelPiS_S_S_S_S_S_iS_S_
    16 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 34 registers, 16 bytes cumulative stack size, 432 bytes cmem[0], 28 bytes cmem[2]


In [17]:
!rm -rf /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw
!./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 0 0
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw


Execution Time for 32 threads and 10 blocks: 3.463296 ms
No errors!
Total Errors : 0	

In [18]:
!ncu --set full ./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 0 0


==PROF== Connected to process 828 (/content/global_queueing)
==PROF== Profiling "bfs_kernel" - 0: 0%....50%....100% - 30 passes
Execution Time for 32 threads and 10 blocks: 1670.822388 ms
==PROF== Disconnected from process 828
[828] global_queueing@127.0.0.1
  bfs_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *) (10, 1, 1)x(32, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.99
    SM Frequency                    Mhz       584.93
    Elapsed Cycles                cycle      399,472
    Memory Throughput                 %         1.70
    DRAM Throughput                   %         1.07
    Duration                         us       682.94
    L1/TEX Cache Throughput           %         6.77
    L2 Cache Throughput   

In [19]:
!./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 0 1
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw


Execution Time for 64 threads and 10 blocks: 3.120736 ms
No errors!
Total Errors : 0	

In [20]:
!ncu --set full ./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 0 1


==PROF== Connected to process 955 (/content/global_queueing)
==PROF== Profiling "bfs_kernel" - 0: 0%....50%....100% - 30 passes
Execution Time for 64 threads and 10 blocks: 1185.221313 ms
==PROF== Disconnected from process 955
[955] global_queueing@127.0.0.1
  bfs_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *) (10, 1, 1)x(64, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         5.00
    SM Frequency                    Mhz       584.87
    Elapsed Cycles                cycle      204,470
    Memory Throughput                 %         3.19
    DRAM Throughput                   %         2.07
    Duration                         us       349.60
    L1/TEX Cache Throughput           %        13.28
    L2 Cache Throughput   

In [21]:
!./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 0 2
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw


Execution Time for 128 threads and 10 blocks: 3.555456 ms
No errors!
Total Errors : 0	

In [22]:
!ncu --set full ./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 0 2


==PROF== Connected to process 1076 (/content/global_queueing)
==PROF== Profiling "bfs_kernel" - 0: 0%....50%....100% - 30 passes
Execution Time for 128 threads and 10 blocks: 1159.676880 ms
==PROF== Disconnected from process 1076
[1076] global_queueing@127.0.0.1
  bfs_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *) (10, 1, 1)x(128, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.98
    SM Frequency                    Mhz       584.79
    Elapsed Cycles                cycle      107,529
    Memory Throughput                 %         6.08
    DRAM Throughput                   %         3.80
    Duration                         us       183.87
    L1/TEX Cache Throughput           %        25.21
    L2 Cache Throughp

In [23]:
!./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 1 0
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw


Execution Time for 32 threads and 25 blocks: 2.868000 ms
No errors!
Total Errors : 0	

In [24]:
!ncu --set full ./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 1 0


==PROF== Connected to process 1193 (/content/global_queueing)
==PROF== Profiling "bfs_kernel" - 0: 0%....50%....100% - 30 passes
Execution Time for 32 threads and 25 blocks: 1163.164307 ms
==PROF== Disconnected from process 1193
[1193] global_queueing@127.0.0.1
  bfs_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *) (25, 1, 1)x(32, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.95
    SM Frequency                    Mhz       584.91
    Elapsed Cycles                cycle      165,403
    Memory Throughput                 %         4.01
    DRAM Throughput                   %         2.53
    Duration                         us       282.78
    L1/TEX Cache Throughput           %         6.78
    L2 Cache Throughput

In [25]:
!./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 1 1
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw


Execution Time for 64 threads and 25 blocks: 2.766112 ms
No errors!
Total Errors : 0	

In [26]:
!ncu --set full ./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 1 1


==PROF== Connected to process 1310 (/content/global_queueing)
==PROF== Profiling "bfs_kernel" - 0: 0%....50%....100% - 30 passes
Execution Time for 64 threads and 25 blocks: 1308.129150 ms
==PROF== Disconnected from process 1310
[1310] global_queueing@127.0.0.1
  bfs_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *) (25, 1, 1)x(64, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         5.03
    SM Frequency                    Mhz       584.76
    Elapsed Cycles                cycle       91,075
    Memory Throughput                 %         7.14
    DRAM Throughput                   %         4.30
    Duration                         us       155.74
    L1/TEX Cache Throughput           %        13.17
    L2 Cache Throughput

In [27]:
!./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 1 2
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw


Execution Time for 128 threads and 25 blocks: 2.689312 ms
No errors!
Total Errors : 0	

In [28]:
!ncu --set full ./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 1 2


==PROF== Connected to process 1433 (/content/global_queueing)
==PROF== Profiling "bfs_kernel" - 0: 0%....50%....100% - 30 passes
Execution Time for 128 threads and 25 blocks: 1272.847290 ms
==PROF== Disconnected from process 1433
[1433] global_queueing@127.0.0.1
  bfs_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *) (25, 1, 1)x(128, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.96
    SM Frequency                    Mhz       584.68
    Elapsed Cycles                cycle       55,943
    Memory Throughput                 %        11.63
    DRAM Throughput                   %         6.66
    Duration                         us        95.68
    L1/TEX Cache Throughput           %        24.34
    L2 Cache Throughp

In [29]:
!./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 2 0
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw


Execution Time for 32 threads and 35 blocks: 3.405568 ms
No errors!
Total Errors : 0	

In [30]:
!ncu --set full ./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 2 0


==PROF== Connected to process 1552 (/content/global_queueing)
==PROF== Profiling "bfs_kernel" - 0: 0%....50%....100% - 30 passes
Execution Time for 32 threads and 35 blocks: 1654.069214 ms
==PROF== Disconnected from process 1552
[1552] global_queueing@127.0.0.1
  bfs_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *) (35, 1, 1)x(32, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         5.01
    SM Frequency                    Mhz       584.85
    Elapsed Cycles                cycle      115,119
    Memory Throughput                 %         5.68
    DRAM Throughput                   %         3.50
    Duration                         us       196.83
    L1/TEX Cache Throughput           %         8.68
    L2 Cache Throughput

In [31]:
!./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 2 1
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw


Execution Time for 64 threads and 35 blocks: 3.400384 ms
No errors!
Total Errors : 0	

In [32]:
!ncu --set full ./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 2 1


==PROF== Connected to process 1671 (/content/global_queueing)
==PROF== Profiling "bfs_kernel" - 0: 0%....50%....100% - 30 passes
Execution Time for 64 threads and 35 blocks: 1315.207153 ms
==PROF== Disconnected from process 1671
[1671] global_queueing@127.0.0.1
  bfs_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *) (35, 1, 1)x(64, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         5.02
    SM Frequency                    Mhz       584.61
    Elapsed Cycles                cycle       66,114
    Memory Throughput                 %         9.82
    DRAM Throughput                   %         5.77
    Duration                         us       113.09
    L1/TEX Cache Throughput           %        14.99
    L2 Cache Throughput

In [33]:
!./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 2 2
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw


Execution Time for 128 threads and 35 blocks: 2.740416 ms
No errors!
Total Errors : 0	

In [34]:
!ncu --set full ./global_queueing /content/drive/MyDrive/ECSE420/lab3/input1.raw /content/drive/MyDrive/ECSE420/lab3/input2.raw /content/drive/MyDrive/ECSE420/lab3/input3.raw /content/drive/MyDrive/ECSE420/lab3/input4.raw /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw 2 2


==PROF== Connected to process 1789 (/content/global_queueing)
==PROF== Profiling "bfs_kernel" - 0: 0%....50%....100% - 30 passes
Execution Time for 128 threads and 35 blocks: 1114.650879 ms
==PROF== Disconnected from process 1789
[1789] global_queueing@127.0.0.1
  bfs_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *) (35, 1, 1)x(128, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.95
    SM Frequency                    Mhz       584.57
    Elapsed Cycles                cycle       43,065
    Memory Throughput                 %        15.11
    DRAM Throughput                   %         8.39
    Duration                         us        73.66
    L1/TEX Cache Throughput           %        23.52
    L2 Cache Throughp

In [35]:
!gcc /content/drive/MyDrive/ECSE420/lab3/compareNextLevelNodes.c -o compareNextLevelNodes

In [36]:
!./compareNextLevelNodes /content/drive/MyDrive/ECSE420/lab3/output_nextLevelNodes_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw

No errors!


In [37]:
!head /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

==> /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw <==
200000
1
0
1
1
1
0
0
0
1

==> /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw <==
200000
1
0
1
1
1
0
0
0
1


In [38]:
!./compareNodeOutput /content/drive/MyDrive/ECSE420/lab3/output_nodeOutput_gq.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

Total Errors : 0	

#Section 3: Block Queueing

In [39]:
!gcc /content/drive/MyDrive/ECSE420/lab3/compareNextLevelNodes.c -o compareNextLevelNodes
!gcc /content/drive/MyDrive/ECSE420/lab3/compareNodeOutput.c -o compareNodeOutput

In [40]:
%%writefile block_queueing.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#include "read_input.h"

#define MAX_NODES 1000000
#define AND 0
#define OR 1
#define NAND 2
#define NOR 3
#define XOR 4
#define XNOR 5

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

// Device Function: Gate Execution
__device__ int execute(int a, int b, int gate) {
    int result;
    switch(gate) {
        case AND: result = a & b; break;
        case OR: result = a | b; break;
        case NAND: result = (a & b) == 0; break;
        case NOR: result = (a | b) == 0; break;
        case XOR: result = a ^ b; break;
        case XNOR: result = (a ^ b) == 0; break;
        default: result = 0;
    }

    if (result == -1) {
        printf("WARNING: execute() returned -1 for Inputs %d, %d, Gate %d\n", a, b, gate);
    }
    return result;
}

__global__ void block_queueing_kernel(int *nodePtrs, int *nodeNeighbors, int *nodeVisited,
                                      int *nodeOutput, int *nodeGate, int *nodeInput,
                                      int *currLevelNodes, int numCurrLevelNodes,
                                      int *nextLevelNodes, int *numNextLevelNodes, int sharedQueueSize) {

    // Initialize shared memory queue
    extern __shared__ int sharedQueue[];
    __shared__ int sharedQueueCounter;

    int tid = threadIdx.x;
    int bid = blockIdx.x;
    int numThreads = blockDim.x;

    if (tid == 0) {
        sharedQueueCounter = 0;
    }

    // Loop over all nodes in the current level
    for (int i = bid; i < numCurrLevelNodes; i += gridDim.x) {
        int node = currLevelNodes[i];
        int start = nodePtrs[node];
        int end = nodePtrs[node + 1];

        // Loop over all neighbors of the node
        for (int j = start + tid; j < end; j += numThreads) {
            int neighbor = nodeNeighbors[j];

            // If the neighbor has not been visited yet
            if (atomicCAS(&nodeVisited[neighbor], 0, 1) == 0) {
                // Update node output
                nodeOutput[neighbor] = execute(nodeOutput[node], nodeInput[neighbor], nodeGate[neighbor]);

                // Add it to the block queue
                int index = atomicAdd(&sharedQueueCounter, 1);
                if (index < sharedQueueSize) {
                    sharedQueue[index] = neighbor;
                }
            }
        }
        __syncthreads();

        // If full, add it to the global queue
        if (tid == 0) {
            while (sharedQueueCounter >= sharedQueueSize) {
                int globalIndex = atomicAdd(numNextLevelNodes, sharedQueueSize);

                // Add to global queue
                for (int k = 0; k < sharedQueueSize; k++) {
                    nextLevelNodes[globalIndex + k] = sharedQueue[k];
                }

                int remaining = sharedQueueCounter - sharedQueueSize;
                for (int k = 0; k < remaining; k++) {
                    sharedQueue[k] = sharedQueue[sharedQueueSize + k];
                }

                sharedQueueCounter = remaining;
            }
        }
        __syncthreads();

        // Add remaining elements to the global queue
        if (tid == 0 && sharedQueueCounter > 0) {
            int globalIndex = atomicAdd(numNextLevelNodes, sharedQueueCounter);
            for (int k = 0; k < sharedQueueCounter; k++) {
                nextLevelNodes[globalIndex + k] = sharedQueue[k];
            }
            sharedQueueCounter = 0;
        }
    }
}


int main(int argc, char **argv)
{
    int numBlock = atoi(argv[1]);
    int blockSize = atoi(argv[2]);
    int sharedQueueSize = atoi(argv[3]);
    char *inputPath1 = argv[4];
    char *inputPath2 = argv[5];
    char *inputPath3 = argv[6];
    char *inputPath4 = argv[7];
    char *outputNodePath = argv[8];
    char *outputNextLevelNodesPath = argv[9];

    int numNodePtrs;
    int numNodes;
    int *nodePtrs_h;
    int *nodeNeighbors_h;
    int *nodeVisited_h;
    int numTotalNeighbors_h;
    int *currLevelNodes_h;
    int numCurrLevelNodes;
    int *nodeGate_h;
    int *nodeInput_h;
    int *nodeOutput_h;

    numNodePtrs = read_input_one_two_four(&nodePtrs_h, inputPath1);
    numTotalNeighbors_h = read_input_one_two_four(&nodeNeighbors_h, inputPath2);
    numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, inputPath3);
    numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, inputPath4);

    int *nextLevelNodes_h = (int *)malloc(numNodes * sizeof(int));
    if (!nextLevelNodes_h)
    {
        perror("Memory allocation failed for nextLevelNodes");
        exit(EXIT_FAILURE);
    }

    int numNextLevelNodes_h = 0;

    // Device Variables
    int *d_nodePtrs, *d_nodeNeighbors, *d_nodeVisited, *d_nodeOutput, *d_nodeGate, *d_nodeInput;
    int *d_currLevelNodes, *d_nextLevelNodes, *d_numNextLevelNodes;

    // Allocate memory on GPU
    checkCudaErr(cudaMalloc(&d_nodePtrs, numNodePtrs * sizeof(int)), "cudaMalloc nodePtrs");
    checkCudaErr(cudaMalloc(&d_nodeNeighbors, numTotalNeighbors_h * sizeof(int)), "cudaMalloc nodeNeighbors");
    checkCudaErr(cudaMalloc(&d_nodeVisited, numNodes * sizeof(int)), "cudaMalloc nodeVisited");
    checkCudaErr(cudaMalloc(&d_nodeOutput, numNodes * sizeof(int)), "cudaMalloc nodeOutput");
    checkCudaErr(cudaMalloc(&d_nodeGate, numNodes * sizeof(int)), "cudaMalloc nodeGate");
    checkCudaErr(cudaMalloc(&d_nodeInput, numNodes * sizeof(int)), "cudaMalloc nodeInput");
    checkCudaErr(cudaMalloc(&d_currLevelNodes, numCurrLevelNodes * sizeof(int)), "cudaMalloc currLevelNodes");
    checkCudaErr(cudaMalloc(&d_nextLevelNodes, numNodes * sizeof(int)), "cudaMalloc nextLevelNodes");
    checkCudaErr(cudaMalloc(&d_numNextLevelNodes, sizeof(int)), "cudaMalloc nextLevelNodes");

    // Copy data from host to device
    checkCudaErr(cudaMemcpy(d_nodePtrs, nodePtrs_h, numNodePtrs * sizeof(int), cudaMemcpyHostToDevice), "cudaMemcpy nodePtrs");
    checkCudaErr(cudaMemcpy(d_nodeNeighbors, nodeNeighbors_h, numTotalNeighbors_h * sizeof(int), cudaMemcpyHostToDevice), "cudaMemcpy nodeNeighbors");
    checkCudaErr(cudaMemcpy(d_nodeVisited, nodeVisited_h, numNodes * sizeof(int), cudaMemcpyHostToDevice), "cudaMemcpy nodeVisited");
    checkCudaErr(cudaMemcpy(d_nodeOutput, nodeOutput_h, numNodes * sizeof(int), cudaMemcpyHostToDevice), "cudaMemcpy nodeOutput");
    checkCudaErr(cudaMemcpy(d_nodeGate, nodeGate_h, numNodes * sizeof(int), cudaMemcpyHostToDevice), "cudaMemcpy nodeGate");
    checkCudaErr(cudaMemcpy(d_nodeInput, nodeInput_h, numNodes * sizeof(int), cudaMemcpyHostToDevice), "cudaMemcpy nodeInput");
    checkCudaErr(cudaMemcpy(d_currLevelNodes, currLevelNodes_h, numCurrLevelNodes * sizeof(int), cudaMemcpyHostToDevice), "cudaMemcpy currLevelNodes");
    checkCudaErr(cudaMemset(d_numNextLevelNodes, 0, sizeof(int)), "cudaMemset nextLevelNodes");

    // CUDA Event Timers
    cudaEvent_t start, stop;
    float milliseconds = 0;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);


    // Launch Kernel with Timing
    cudaEventRecord(start);

    // Execute the kernel
    block_queueing_kernel<<<numBlock, blockSize, sharedQueueSize * sizeof(int)>>>(
        d_nodePtrs, d_nodeNeighbors, d_nodeVisited, d_nodeOutput, d_nodeGate,
        d_nodeInput, d_currLevelNodes, numCurrLevelNodes, d_nextLevelNodes,
        d_numNextLevelNodes, sharedQueueSize);

    checkCudaErr(cudaDeviceSynchronize(), "CUDA synchronize");
    checkCudaErr(cudaGetLastError(), "CUDA error");

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);

    printf("Execution time: %.3f ms\n", milliseconds);

    // Copy results back to host
    checkCudaErr(cudaMemcpy(nextLevelNodes_h, d_nextLevelNodes, numNodes * sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy nextLevelNodes");
    checkCudaErr(cudaMemcpy(&numNextLevelNodes_h, d_numNextLevelNodes, sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy numNextLevelNodes");
    checkCudaErr(cudaMemcpy(nodeOutput_h, d_nodeOutput, numNodes * sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy nodeOutput");
    checkCudaErr(cudaMemcpy(nodeVisited_h, d_nodeVisited, numNodes * sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy nodeVisited");
    write_output_file(outputNodePath, nodeOutput_h, numNodes);
    write_output_file(outputNextLevelNodesPath, nextLevelNodes_h, numNextLevelNodes_h);

    // Clean up
    checkCudaErr(cudaFree(d_nodePtrs), "cudaFree nodePtrs");
    checkCudaErr(cudaFree(d_nodeNeighbors), "cudaFree nodeNeighbors");
    checkCudaErr(cudaFree(d_nodeVisited), "cudaFree nodeVisited");
    checkCudaErr(cudaFree(d_nodeOutput), "cudaFree nodeOutput");
    checkCudaErr(cudaFree(d_nodeGate), "cudaFree nodeGate");
    checkCudaErr(cudaFree(d_nodeInput), "cudaFree nodeInput");
    checkCudaErr(cudaFree(d_currLevelNodes), "cudaFree currLevelNodes");
    checkCudaErr(cudaFree(d_nextLevelNodes), "cudaFree nextLevelNodes");
    checkCudaErr(cudaFree(d_numNextLevelNodes), "cudaFree nextLevelNodes");

    free(nodePtrs_h);
    free(nodeNeighbors_h);
    free(nodeVisited_h);
    free(nodeGate_h);
    free(nodeInput_h);
    free(nodeOutput_h);

    return 0;
}

Writing block_queueing.cu


In [41]:
!rm block_queueing
!rm -rf output_nextLevelNodes_bq.raw
!rm -rf output_nodeOutput_bq.raw
!nvcc --ptxas-options=-v -arch=sm_75 -gencode=arch=compute_75,code=sm_75 block_queueing.cu -o block_queueing

rm: cannot remove 'block_queueing': No such file or directory
ptxas info    : 59 bytes gmem, 16 bytes cmem[4]
ptxas info    : Compiling entry function '_Z21block_queueing_kernelPiS_S_S_S_S_S_iS_S_i' for 'sm_75'
ptxas info    : Function properties for _Z21block_queueing_kernelPiS_S_S_S_S_S_iS_S_i
    16 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 44 registers, 16 bytes cumulative stack size, 16 bytes smem, 436 bytes cmem[0], 28 bytes cmem[2]


In [42]:
!./block_queueing 25 32 32 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_25_32_32.raw \
output_nextLevelNodes_bq_25_32_32.raw

!./compareNextLevelNodes output_nextLevelNodes_bq_25_32_32.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput output_nodeOutput_bq_25_32_32.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

Execution time: 4.255 ms
No errors!
Total Errors : 0	

In [43]:
!./block_queueing 25 32 64 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_25_32_64.raw \
output_nextLevelNodes_bq_25_32_64.raw

!./compareNextLevelNodes output_nextLevelNodes_bq_25_32_64.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput output_nodeOutput_bq_25_32_64.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

Execution time: 4.846 ms
No errors!
Total Errors : 0	

In [44]:
!./block_queueing 25 64 32 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_25_64_32.raw \
output_nextLevelNodes_bq_25_64_32.raw

!./compareNextLevelNodes output_nextLevelNodes_bq_25_64_32.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput output_nodeOutput_bq_25_64_32.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

Execution time: 9.050 ms
No errors!
Total Errors : 0	

In [45]:
!./block_queueing 25 64 64 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_25_64_64.raw \
output_nextLevelNodes_bq_25_64_64.raw

!./compareNextLevelNodes output_nextLevelNodes_bq_25_64_64.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput output_nodeOutput_bq_25_64_64.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

Execution time: 4.057 ms
No errors!
Total Errors : 0	

In [46]:
!./block_queueing 35 32 32 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_35_32_32.raw \
output_nextLevelNodes_bq_35_32_32.raw

!./compareNextLevelNodes output_nextLevelNodes_bq_35_32_32.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput output_nodeOutput_bq_35_32_32.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

Execution time: 3.822 ms
No errors!
Total Errors : 0	

In [47]:
!./block_queueing 35 32 64 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_35_32_64.raw \
output_nextLevelNodes_bq_35_32_64.raw

!./compareNextLevelNodes output_nextLevelNodes_bq_35_32_64.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput output_nodeOutput_bq_35_32_64.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

Execution time: 4.395 ms
No errors!
Total Errors : 0	

In [48]:
!./block_queueing 35 64 32 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_35_64_32.raw \
output_nextLevelNodes_bq_35_64_32.raw

!./compareNextLevelNodes output_nextLevelNodes_bq_35_64_32.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput output_nodeOutput_bq_35_64_32.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

Execution time: 4.190 ms
No errors!
Total Errors : 0	

In [49]:
!./block_queueing 35 64 64 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_35_64_64.raw \
output_nextLevelNodes_bq_35_64_64.raw

!./compareNextLevelNodes output_nextLevelNodes_bq_35_64_64.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput output_nodeOutput_bq_35_64_64.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

Execution time: 4.217 ms
No errors!
Total Errors : 0	

In [50]:
!ncu --set full ./block_queueing 25 32 32 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_25_32_32.raw \
output_nextLevelNodes_bq_25_32_32.raw

==PROF== Connected to process 2321 (/content/block_queueing)
==PROF== Profiling "block_queueing_kernel" - 0: 0%....50%....100% - 30 passes
Execution time: 1368.458 ms
==PROF== Disconnected from process 2321
[2321] block_queueing@127.0.0.1
  block_queueing_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *, int) (25, 1, 1)x(32, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         5.00
    SM Frequency                    Mhz       584.98
    Elapsed Cycles                cycle      949,891
    Memory Throughput                 %         1.42
    DRAM Throughput                   %         0.62
    Duration                         ms         1.62
    L1/TEX Cache Throughput           %         2.27
    L2 Cache Throughput       

In [51]:
!ncu --set full ./block_queueing 25 32 64 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_25_32_64.raw \
output_nextLevelNodes_bq_25_32_64.raw

!./compareNextLevelNodes output_nextLevelNodes_bq_25_32_64.raw /content/drive/MyDrive/ECSE420/lab3/sol_nextLevelNodes.raw
!./compareNodeOutput output_nodeOutput_bq_25_32_64.raw /content/drive/MyDrive/ECSE420/lab3/sol_nodeOutput.raw

==PROF== Connected to process 2395 (/content/block_queueing)
==PROF== Profiling "block_queueing_kernel" - 0: 0%....50%....100% - 30 passes
Execution time: 1359.730 ms
==PROF== Disconnected from process 2395
[2395] block_queueing@127.0.0.1
  block_queueing_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *, int) (25, 1, 1)x(32, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         5.00
    SM Frequency                    Mhz       584.97
    Elapsed Cycles                cycle      950,125
    Memory Throughput                 %         1.42
    DRAM Throughput                   %         0.62
    Duration                         ms         1.62
    L1/TEX Cache Throughput           %         2.27
    L2 Cache Throughput       

In [52]:
!ncu --set full ./block_queueing 25 64 32 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_25_64_32.raw \
output_nextLevelNodes_bq_25_64_32.raw

==PROF== Connected to process 2509 (/content/block_queueing)
==PROF== Profiling "block_queueing_kernel" - 0: 0%....50%....100% - 30 passes
Execution time: 1458.159 ms
==PROF== Disconnected from process 2509
[2509] block_queueing@127.0.0.1
  block_queueing_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *, int) (25, 1, 1)x(64, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.99
    SM Frequency                    Mhz       584.96
    Elapsed Cycles                cycle      812,715
    Memory Throughput                 %         1.96
    DRAM Throughput                   %         0.72
    Duration                         ms         1.39
    L1/TEX Cache Throughput           %         3.15
    L2 Cache Throughput       

In [53]:
!ncu --set full ./block_queueing 25 64 64 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_25_64_64.raw \
output_nextLevelNodes_bq_25_64_64.raw

==PROF== Connected to process 2585 (/content/block_queueing)
==PROF== Profiling "block_queueing_kernel" - 0: 0%....50%....100% - 30 passes
Execution time: 1355.391 ms
==PROF== Disconnected from process 2585
[2585] block_queueing@127.0.0.1
  block_queueing_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *, int) (25, 1, 1)x(64, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.99
    SM Frequency                    Mhz       584.97
    Elapsed Cycles                cycle      812,859
    Memory Throughput                 %         1.96
    DRAM Throughput                   %         0.72
    Duration                         ms         1.39
    L1/TEX Cache Throughput           %         3.15
    L2 Cache Throughput       

In [54]:
!ncu --set full /block_queueing 35 32 32 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_35_32_32.raw \
output_nextLevelNodes_bq_35_32_32.raw

==PROF== Connected to process 2655 (/content/block_queueing)
==PROF== Profiling "block_queueing_kernel" - 0: 0%....50%....100% - 30 passes
Execution time: 1335.010 ms
==PROF== Disconnected from process 2655
[2655] block_queueing@127.0.0.1
  block_queueing_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *, int) (35, 1, 1)x(32, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.99
    SM Frequency                    Mhz       584.97
    Elapsed Cycles                cycle      680,252
    Memory Throughput                 %         1.98
    DRAM Throughput                   %         0.86
    Duration                         ms         1.16
    L1/TEX Cache Throughput           %         3.12
    L2 Cache Throughput       

In [55]:
!ncu --set full ./block_queueing 35 32 64 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_35_32_64.raw \
output_nextLevelNodes_bq_35_32_64.raw

==PROF== Connected to process 2731 (/content/block_queueing)
==PROF== Profiling "block_queueing_kernel" - 0: 0%....50%....100% - 30 passes
Execution time: 1756.641 ms
==PROF== Disconnected from process 2731
[2731] block_queueing@127.0.0.1
  block_queueing_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *, int) (35, 1, 1)x(32, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         5.00
    SM Frequency                    Mhz       584.96
    Elapsed Cycles                cycle      680,389
    Memory Throughput                 %         1.98
    DRAM Throughput                   %         0.86
    Duration                         ms         1.16
    L1/TEX Cache Throughput           %         3.12
    L2 Cache Throughput       

In [56]:
!ncu --set full ./block_queueing 35 64 32 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_35_64_32.raw \
output_nextLevelNodes_bq_35_64_32.raw

==PROF== Connected to process 2805 (/content/block_queueing)
==PROF== Profiling "block_queueing_kernel" - 0: 0%....50%....100% - 30 passes
Execution time: 1290.202 ms
==PROF== Disconnected from process 2805
[2805] block_queueing@127.0.0.1
  block_queueing_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *, int) (35, 1, 1)x(64, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         5.00
    SM Frequency                    Mhz       584.95
    Elapsed Cycles                cycle      581,503
    Memory Throughput                 %         2.74
    DRAM Throughput                   %         1.01
    Duration                         us       994.11
    L1/TEX Cache Throughput           %         4.08
    L2 Cache Throughput       

In [57]:
!ncu --set full ./block_queueing 35 64 64 \
/content/drive/MyDrive/ECSE420/lab3/input1.raw \
/content/drive/MyDrive/ECSE420/lab3/input2.raw \
/content/drive/MyDrive/ECSE420/lab3/input3.raw \
/content/drive/MyDrive/ECSE420/lab3/input4.raw \
output_nodeOutput_bq_35_64_64.raw \
output_nextLevelNodes_bq_35_64_64.raw

==PROF== Connected to process 2875 (/content/block_queueing)
==PROF== Profiling "block_queueing_kernel" - 0: 0%....50%....100% - 30 passes
Execution time: 1312.479 ms
==PROF== Disconnected from process 2875
[2875] block_queueing@127.0.0.1
  block_queueing_kernel(int *, int *, int *, int *, int *, int *, int *, int, int *, int *, int) (35, 1, 1)x(64, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.99
    SM Frequency                    Mhz       584.95
    Elapsed Cycles                cycle      581,790
    Memory Throughput                 %         2.74
    DRAM Throughput                   %         1.02
    Duration                         us       994.59
    L1/TEX Cache Throughput           %         4.08
    L2 Cache Throughput       